Code can be divided into a few parts....

1. Combine 
2. Changing mask pixel values (labels) from 4 to 3 (as the original labels are 0, 1, 2, 4)
3. Visualize


https://pypi.org/project/nibabel/

All BraTS multimodal scans are available as NIfTI files (.nii.gz) -> commonly used medical imaging format to store brain imagin data obtained using MRI and describe different MRI settings

- T1    - T1-weighted, native image, sagittal or axial 2D acquisitions, with 1-6 mm slice thickness.
- T1c   - T1-weighted, contrast-enhanced (Gadolinium) image, with 3D acquisition and 1 mm isotropic voxel size for most patients.
- T2    - T2-weighted image, axial 2D acquisition, with 2-6 mm slice thickness.
- FLAIR - T2-weighted FLAIR image, axial, coronal, or sagittal 2D acquisitions, 2-6 mm slice thickness.

Note: Segmented file name in Folder 355 has a weird name. Rename it to match others.

In [1]:
import numpy as np
import nibabel as nib
import glob
from tensorflow.keras.utils import to_categorical       # Since it's a multi-class semantic segmentation we need to convert images into categorical 
import matplotlib.pyplot as plt
from tifffile import imwrite             # To save as tiff files 

from sklearn.preprocessing import MinMaxScaler          # To scale all values
scaler = MinMaxScaler()

### Getting an initial understanding of the dataset

In [ ]:
# PART 1: Load sample images and visualize
#Includes, dividing each image by its max to scale them to [0,1]
#Converting mask from float to uint8
#Changing mask pixel values (labels) from 4 to 3 (as the original labels are 0, 1, 2, 4)
#Visualize


# View a few images

# Note: Segmented file name in Folder 355 had a weird name. Renamed it to match others.

TRAIN_DATASET_PATH = 'BraTS_2020_dataset/MICCAI_BraTS2020_TrainingData/'
#VALIDATION_DATASET_PATH = 'BraTS2020_ValidationData/MICCAI_BraTS2020_ValidationData'

In [ ]:
# Load a single image as a numpy array
test_image_flair = nib.load(TRAIN_DATASET_PATH + 'BraTS20_Training_355/BraTS20_Training_355_flair.nii').get_fdata()

print(test_image_flair.max())

### Normalize NIfTI images to values btw 0 to 1 using MinMaxScaler

In [ ]:
# Scalers are applied to 1D so let us reshape and then reshape back to original shape, i.e flattening then reforming
test_image_flair = scaler.fit_transform(test_image_flair.reshape(-1, test_image_flair.shape[-1])).reshape(test_image_flair.shape)

In [ ]:
test_image_t1 = nib.load(TRAIN_DATASET_PATH + 'BraTS20_Training_355/BraTS20_Training_355_t1.nii').get_fdata()

test_image_t1 = scaler.fit_transform(test_image_t1.reshape(-1, test_image_t1.shape[-1])).reshape(test_image_t1.shape)

In [ ]:
test_image_t1ce = nib.load(TRAIN_DATASET_PATH + 'BraTS20_Training_355/BraTS20_Training_355_t1ce.nii').get_fdata()

test_image_t1ce = scaler.fit_transform(test_image_t1ce.reshape(-1, test_image_t1ce.shape[-1])).reshape(test_image_t1ce.shape)

In [ ]:
test_image_t2 = nib.load(TRAIN_DATASET_PATH + 'BraTS20_Training_355/BraTS20_Training_355_t2.nii').get_fdata()

test_image_t2 = scaler.fit_transform(test_image_t2.reshape(-1, test_image_t2.shape[-1])).reshape(test_image_t2.shape)

In [ ]:
test_mask = nib.load(TRAIN_DATASET_PATH + 'BraTS20_Training_355/BraTS20_Training_355_seg.nii').get_fdata()

# Convert mask to type o funsigned integer 8 coz all we have is values 0, 1, 2, 4
test_mask = test_mask.astype(np.uint8)

In [ ]:
print(np.unique(test_mask))             # 0, 1, 2, 4 (Need to re-encode to 0, 1, 2, 3)

In [ ]:
# Reassign mask values from 4 to 3
test_mask[test_mask == 4] = 3

print(np.unique(test_mask)) 

In [ ]:
# Randomly selecting a slice of brain & plotting 
import random

n_slice=random.randint(0, test_mask.shape[2])

plt.figure(figsize=(12, 8))

plt.subplot(231)
plt.imshow(test_image_flair[:,:,n_slice], cmap='gray')
plt.title('Image flair')

plt.subplot(232)
plt.imshow(test_image_t1[:,:,n_slice], cmap='gray')
plt.title('Image t1')

plt.subplot(233)
plt.imshow(test_image_t1ce[:,:,n_slice], cmap='gray')
plt.title('Image t1ce')

plt.subplot(234)
plt.imshow(test_image_t2[:,:,n_slice], cmap='gray')
plt.title('Image t2')

plt.subplot(235)
plt.imshow(test_mask[:,:,n_slice])
plt.title('Mask')


plt.show()

# rerun this cell till u get satisfacory images

### Combine t1ce, t2, and flair into single numpy array (multichannel image)

In [ ]:
# PART 2: Explore the process of combining images to channels and divide them to patches
# Includes...
# Combining all 4 images to 4 channels of a numpy array so that we can train it later
# Flair, T1CE, annd T2 have the most information

combined_x = np.stack([test_image_flair, test_image_t1ce, test_image_t2], axis=3)

# t1 ko bhi add karke test kar sakte hai

In [ ]:
# Crop to a size to be divisible by 64 so we can later extract 64x64x64 patches. 
# cropping x, y, and z
# combined_x = combined_x[24:216, 24:216, 13:141]

combined_x = combined_x[56:184, 56:184, 13:141] #Crop to 128x128x128x4

In [ ]:
# Do the same for mask
test_mask = test_mask[56:184, 56:184, 13:141]

In [ ]:
# Sanity check
n_slice = random.randint(0, test_mask.shape[2])

plt.figure(figsize=(12, 8))

plt.subplot(221)
plt.imshow(combined_x[:,:,n_slice, 0], cmap='gray')
plt.title('Image flair')

plt.subplot(222)
plt.imshow(combined_x[:,:,n_slice, 1], cmap='gray')
plt.title('Image t1ce')

plt.subplot(223)
plt.imshow(combined_x[:,:,n_slice, 2], cmap='gray')
plt.title('Image t2')

plt.subplot(224)
plt.imshow(test_mask[:,:,n_slice])
plt.title('Mask')

plt.show()

In [ ]:
# imsave('BraTS2020_TrainingData/combined255.tif', combined_x)

# Instead of saving as a tiff file, prefer ot save as a numpy array

np.save('BraTS2020_TrainingData/combined255.npy', combined_x)

# Verify image is being read properly
# my_img=imread('BraTS_2020_dataset/combined255.tif')

In [ ]:
# Load image
my_img = np.load('BraTS2020_TrainingData/combined255.npy')

In [ ]:
# Convert mask to categorical
test_mask = to_categorical(test_mask, num_classes=4)

In [ ]:
# Now let us apply the same as above to all the images...
# Merge channels, crop, patchify, save
# GET DATA READY =  GENERATORS OR OTHERWISE

# Keras datagenerator does not support 3d


# images lists harley

#t1_list = sorted(glob.glob('BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/*/*t1.nii'))              # t1_list is not req

t2_list = sorted(glob.glob('BraTS_2020_dataset/MICCAI_BraTS2020_TrainingData/*/*t2.nii'))

t1ce_list = sorted(glob.glob('BraTS_2020_dataset/MICCAI_BraTS2020_TrainingData/*/*t1ce.nii'))

flair_list = sorted(glob.glob('BraTS_2020_dataset/MICCAI_BraTS2020_TrainingData/*/*flair.nii'))

mask_list = sorted(glob.glob('BraTS_2020_dataset/MICCAI_BraTS2020_TrainingData/*/*seg.nii'))

In [ ]:
# Each volume generates 18 64x64x64x4 sub-volumes. 
# Total 369 volumes = 6642 sub volumes

for img in range(len(t2_list)):             # Using t1_list as all lists are of same size
    
    print("Now preparing image and masks number: ", img)
      
    temp_image_t2 = nib.load(t2_list[img]).get_fdata()
    temp_image_t2 = scaler.fit_transform(temp_image_t2.reshape(-1, temp_image_t2.shape[-1])).reshape(temp_image_t2.shape)
   
    temp_image_t1ce = nib.load(t1ce_list[img]).get_fdata()
    temp_image_t1ce = scaler.fit_transform(temp_image_t1ce.reshape(-1, temp_image_t1ce.shape[-1])).reshape(temp_image_t1ce.shape)
   
    temp_image_flair = nib.load(flair_list[img]).get_fdata()
    temp_image_flair = scaler.fit_transform(temp_image_flair.reshape(-1, temp_image_flair.shape[-1])).reshape(temp_image_flair.shape)
        

    temp_mask = nib.load(mask_list[img]).get_fdata()
    temp_mask = temp_mask.astype(np.uint8)
    temp_mask[temp_mask == 4] = 3         # Reassign mask values 4 to 3
    # print(np.unique(temp_mask))
    
    
    # Stack flair, t1ce & t2 together
    temp_combined_images = np.stack([temp_image_flair, temp_image_t1ce, temp_image_t2], axis = 3)
    
    # Crop to a size to be divisible by 64 so we can later extract 64x64x64 patches. 
    # cropping x, y, and z
    temp_combined_images=temp_combined_images[56:184, 56:184, 13:141]
    temp_mask = temp_mask[56:184, 56:184, 13:141]
    
    
    # Finding volumes with segmented vol > 1%
    
    val, counts = np.unique(temp_mask, return_counts=True)
    
    # At least 1% useful volume with labels that are not 0
    if (1 - (counts[0]/counts.sum())) > 0.01:  

        print("Save Me")
        temp_mask= to_categorical(temp_mask, 
        num_classes=4)
        
        np.save('BraTS_2020_dataset/input_data_3channels/images/image_' + str(img)+'.npy', temp_combined_images)
        
        np.save('BraTS_2020_dataset/input_data_3channels/masks/mask_' + str(img) + '.npy', temp_mask)
        
    else:
        print("I am useless")   

Jab tu code run kar rha hoga toh instead of splitting training data into train, test & val, use the separate validation folder 

In [ ]:
# Repeat the same from above for validation data folder (prefer this) OR Split training data into train and validation (not this, although DigitalSreeni did this for faster training of model)

"""
Code for splitting folder into train, test, and val.
Once the new folders are created rename them and arrange in the format below to be used
for semantic segmentation using data generators. 

pip install split-folders
"""
# import splitfolders         # or import split_folders

# input_folder = 'BraTS2020_TrainingData/input_data_3channels/'
# output_folder = 'BraTS2020_TrainingData/input_data_128/'

# # Split with a ratio.
# # To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
# splitfolders.ratio(input_folder, output=output_folder, seed=42, ratio=(.75, .25), group_prefix=None) # default values